# MNIST in pytorch with Artificial Neural Network

In [1]:
import numpy as np
import scipy.stats
import scipy.special

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
%matplotlib inline

## Standard boilerplate to import torch and torch related modules
import torch

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

The [*MNIST* dataset](https://en.wikipedia.org/wiki/MNIST_database) is one of the classic datasets in Machine Learning and is often one of the first datasets against which new classification algorithms test themselves.  It consists of 70,000 images of handwritten digits, each of which is 28x28 pixels. You will be using PyTorch to build a handwritten digit classifier that you will train, validate, and test with MNIST. 



In [2]:
# Regression Parent Class
class Regression(object):
    
    def __init__(self):
        self.params = dict()
    
    def get_params(self, k):
        return self.params.get(k, None)
    
    def set_params(self, **kwargs):
        for k,v in kwargs.items():
            self.params[k] = v
        
                    
    def fit(self, X, y):
        raise NotImplementedError()
        
    def predict(self, X):
        raise NotImplementedError()
        
    def score(self, X, y):
        raise NotImplementedError()
        

In [3]:
def load_data(validation_split=10000, batch_size=256):
        """load the MNIST training and test sets from MNIST"""
        
        
        ## We start by defining our training dataset
        ## --root-- a string pointing to the relative path of the directory where we'll store our MNIST data
        ## --train-- tells us whether to download the training set (True) or the test set (False)
        ## MNIST in torchvision only has train (60K) and test (10K) datasets.  Other datasets also have a validation set
        ## --transforms-- is a torchvision.transforms object that specifies what transforms to apply to each element
        ## in the dataset.  The required transform is transforms.ToTensor() that turns each element into a PyTorch floating
        ## point tensor object.  You could also add others like transforms.Normalize if you wished
        ## --download-- specifies whether to download the data from the online urls.   If set to false, then you should
        ## provide the data locally yourself
        train_dataset = datasets.MNIST(root='./data',
                                    train=True,
                                    transform=transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,)),
                                     ]),
                                    download=True)

        ## similar to the above, the main difference is that we should set train=False since we want the

        ## test set data
        test_dataset = datasets.MNIST(root='./data',
                                   train=False,
                                   transform=transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,)),
                                     ]),
                                   download=True)

        ## A DataLoader or (Dataset Loader) turns the specified data set into a sequence of data elements
        ## that you can access in your loops for training or evaluating accuracy, etc.

        ## First we need to further split our training dataset into training and validation sets.

        # Define the indices
        indices = list(range(len(train_dataset))) # start with all the indices in training set

        # Define your batch_size
        batch_size = batch_size

        # Random, non-contiguous split
        validation_idx = np.random.choice(indices, size=validation_split, replace=False)
        train_idx = list(set(indices) - set(validation_idx))

        # define our samplers -- we use a SubsetRandomSampler because it will return
        # a random subset of the split defined by the given indices without replacement
        train_sampler = SubsetRandomSampler(train_idx)
        validation_sampler = SubsetRandomSampler(validation_idx)

        # Create the train_loader -- use your real batch_size which you
        # I hope have defined somewhere above
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                        batch_size=batch_size, sampler=train_sampler)

        # You can use your above batch_size or just set it to 1 here.  Your validation
        # operations shouldn't be computationally intensive or require batching.
        validation_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                        batch_size=validation_split, sampler=validation_sampler)

        test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                  batch_size=len(test_dataset),
                                                  shuffle=False)

        return (train_dataset, test_dataset, 
                train_loader, test_loader, 
                validation_loader,
                train_idx, validation_idx
               )

In [4]:
train_dataset, test_dataset, train_loader, test_loader, validation_loader,train_idx, validation_idx = load_data()

In [5]:
## Stolen from excellent visualization from submission from Madeleine Duran/Sarah Walker
def viz_training_loss(training_losses):
    """Visualize/Plot our training loss"""
    epochs = len(training_losses)
    fig, axes = plt.subplots(nrows=1, ncols=epochs, figsize=(20,5), sharex=True, sharey=True)
    plt.suptitle("Loss Trajectory for MNIST LR Model", fontsize=20, weight='heavy')
    for i in range(epochs):
        axes[i].plot(range(len(training_losses[i])), training_losses[i])
        axes[i].set_title("epoch {}".format(i))
        if i % 2 == 1:
            axes[i].axvspan(0, len(training_losses[i]), facecolor='gray', alpha=0.2)
    plt.subplots_adjust(wspace=0)

## MLP



The multilayer perceptron can be understood as a logistic regression classifier in which the input is first transformed using a learnt non-linear transformation. The non-linear transformation is often chosen to be either the logistic function or the $\tanh$ function or the RELU function, and its purpose is to project the data into a space where it becomes linearly separable. The output of this so-called hidden layer is then passed to the logistic regression graph that we have constructed in the first problem. 

![](http://deeplearning.net/tutorial/_images/mlp.png)

We'll construct a model with **1 hidden layer**. That is, you will have an input layer, then a hidden layer with the nonlinearity, and finally an output layer with cross-entropy loss (or equivalently log-softmax activation with a negative log likelihood loss).

2.1. Using a similar architecture as in Question 1 and the same training, validation and test sets, build a PyTorch model for the multilayer perceptron. Use the $\tanh$ function as the non-linear activation function. 

2.2. The initialization of the weights matrix for the hidden layer must assure that the units (neurons) of the perceptron operate in a regime where information gets propagated. For the $\tanh$ function, you may find it advisable to initialize with the interval $\left[-\sqrt{\frac{6}{fan_{in}+fan_{out}}},\sqrt{\frac{6}{fan_{in}+fan_{out}}}\right]$, where $fan_{in}$ is the number of units in the $(i-1)$-th layer, and $fan_{out}$ is the number of units in the i-th layer.  This is known as **Xavier Initialization**.  Use Xavier Initialization to initialize your MLP.  Feel free to use PyTorch's in-built Xavier Initialization methods.

2.3. Using $\lambda = 0.01$ to compare with Question 1, experiment with the learning rate (try 0.1 and 0.01 for example), batch size (use 64, 128 and 256) and the number of units in your hidden layer (use between 25 and 200 units). For what combination of these parameters do you obtain the highest validation accuracy?  You may want to start with 20 epochs for running time and experiment a bit to make sure that your models reach convergence. 

2.4. For your best combination plot the cross-entropy loss on the training set as a function of iteration.

2.5. For your best combination use classification accuracy to evaluate how well your model is performing on the validation set at the end of each epoch. Plot this validation accuracy as the model trains.

2.6. Select what you consider the best set of parameters and predict the labels of the test set. Compare your predictions with the given labels. What classification accuracy do you obtain on the training and test sets?

2.7. How does your test accuracy compare to that of the logistic regression classifier in Question 1?  Compare best parameters for both models.

2.8. What classes are most likely to be misclassified? Plot some misclassified training and test set images.



In [6]:

class ANN(Regression):
    
    def __init__(self, input_model, reg_rate = 0.01, learning_rate=0.1, batch_size=256, epochs=30, hidden=None):
        
        super().__init__()
        
        
        ## Load MNIST Data
        #train_dataset, test_dataset, train_loader, test_loader, validation_loader, train_idx, validation_idx = self.load_data(batch_size=batch_size)
        
        ## Add Datasets and Data Loaders to our params
        self.set_params(train_dataset=train_dataset, 
                        train_loader=train_loader,
                        test_dataset=test_dataset,
                        test_loader=test_loader,
                        validation_loader=validation_loader,
                        train_idx=train_idx,
                        validation_idx=validation_idx
                       )
        
        
        ## Here we instantiate the PyTorch model that we so nicely defined previously
        if hidden == None:
            model = input_model()
        else:
            model = input_model(hidden=hidden)

        ## Here we define our loss function.  We're using CrossEntropyLoss but other options include
        ## NLLLoss (negative log likelihood loss for when the log_softmax activation is explicitly defined
        ## on the output layer), MSELoss for OLS Regression, KLLDivLoss for KL Divergence, BCELoss
        ## for binary cross entropy and many others
        criterion = torch.nn.CrossEntropyLoss()

        ## Here we define our optimizer.  In class we've been using SGD although in practice one will often
        ## use other optimizers like Adam or RMSProp.  The primary parameter the optimizer takes is the
        ## set of parameters in your model.  Fortunately those are easily accessible via model.paramters()
        ## where model is the instance of the model you defined.  Other useful parameters include lr for the
        ## learning rate and weight_decay for the rate of l2 regularization.
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=reg_rate)
        
        
        ## Set the rest of our parameters -- batch_size, learning_rate, epochs, optimizer,
        ## model and criterion
        
        ## Add Datasets and Data Loaders to our params
        self.set_params(optimizer=optimizer, 
                        learning_rate=learning_rate,
                        weight_decay=reg_rate,
                        batch_size=batch_size,
                        model=model,
                        criterion=criterion,
                        epochs=epochs)   
    

        
    def get_loader(self, dataset):
        """Retrieve dataloader, images, labels based upon dataset name"""
        
        if dataset == 'Test':
            loader = self.get_params('test_loader')
        elif dataset == 'Validation':
            loader = self.get_params('validation_loader')
        else:
            loader = self.get_params('train_loader')
            
        # Get Loader
        return loader
    
    def predict(self, dataset='Test'):
        """Classify images based on the fitted logistic regression model"""

        loader = self.get_loader(dataset)
        
        predictions = []
        all_labels = []
        misclassified = []
        misclassified_images = []
        misclassified_labels = []
        misclassified_preds = np.array([])
        correct = 0
        model = self.get_params('model')

        for inputs, labels in loader:

            ## get the inputs from the dataloader and turn into a variable for 
            ## feeding to the model
            inputs = Variable(inputs)

            ## Reshape so that batches work properly
            inputs = inputs.view(-1, 28*28)

            # run our model on the inputs
            outputs = model(inputs)

            # get the class of the max log-probability
            pred = outputs.data.max(1)[1]
            
            # get the correct predictions
            correct += (pred == labels).sum()

            # save current batch of predictions
            predictions += list(pred)
            
            # save all labels
            all_labels += list(labels)
                  
        return np.array(predictions), correct.item()
    
    
    def score(self, dataset='Test', print_score=True):
        """Calculate accuracy score based upon model classification"""
        
        preds, correct = self.predict(dataset=dataset)
        #correct = self.get_params('correct_predictions')
        total = len(preds)
        
        if print_score:
            print('Dataset: {} \nAccuracy: {}/{} ({:.1f}%)\n'.format(
                dataset, correct, total, 100.0 * correct / total))
        #print(">>>", correct/total, type(correct), type(total))
        return correct/total
    
        
    def fit(self, do_validation=True, show_validation=True):
        """Fit our logistic regression model on MNIST training set"""
        
        ## We defined a number of variables in our constructor -- let's reclaim them here
        optimizer=self.get_params("optimizer")
        model=self.get_params("model")
        epochs=self.get_params("epochs")
        criterion=self.get_params("criterion")
        train_loader=self.get_params("train_loader")
        
        ## Get the Total size of training set
        self.get_params('train_dataset')
        training_size = len(self.get_params('train_idx'))
        
        iterations = int(np.ceil(training_size/self.get_params("batch_size")))
        
        ## We need something to keep track of our losses
        losses = np.zeros((epochs, iterations))
        
        ## We need something ot keep track of our validation scores
        validation_scores = np.zeros(epochs)
  
        
        ## Our training loop.  We can loop over a fixed number of epochs or
        ## using a sensitivity parameter (i.e. until net change in loss is
        ## below a certain tolerance).  Here we iterate over a fixed number of
        ## epochs
        for epoch in range(epochs):

            ## We defined our train_loader DataLoader earlier.  The train_loader is a
            ## sequence of tuples with the first element of each tuple being
            ## the batched training inputs (the batch_size being defined in your DataLoader)
            ## and the second second element of each tuple being the corresponding labels
            ## more or less all the pytorch classes are built to handle batching transparently

            ## loop through the DataLoader.  Each loop is one iteration.  All the loops
            ## form one epoch
            for batch_index, (inputs, labels) in enumerate(train_loader):

                # Convert the inputs/labels passed from the DataLoader into
                # autograd Variables.  The dataloader provides them as PyTorch Tensors
                # per the transforms.ToTensor() operation.
                inputs, labels = Variable(inputs), Variable(labels)

                ## as mentioned above we receive the inputs as tensors of size (batch_size,1, 28, 28)
                ## which is effectively (batch_size, 28, 28) basically as a 3 dimensional tensor
                ## representing a stack of (28x28) matrices with each matrix element a floating point number
                ## representing the value of that pixel in the image.  Unfortunately our Neural Network model
                ## can't handle that representation and needs a pixel matrices to be flattened into a row vector
                ## of inputs.  The model takes a 2d tensor representing batch of such row vectors each row vector
                ## representing one set of inputs corresponding to one image.  In order to accomplish this
                ## flattening we use the .view method defined on autograd Variables.
                inputs = inputs.view(-1, 28*28)

                # we need to zero out our gradients after each pass
                optimizer.zero_grad()


                ## This is the optimize - forward step - backwards step part of our design pattern

                # this is the forward step --> we calculate the new outputs based upon the input data from
                # this batch and store the outputs in a variable
                outputs = model(inputs)

                # we compare the outputs to the ground truth labels in the batch to calculate the loss for this step
                loss = criterion(outputs, labels)
                
                ## count the loss
                losses[epoch,batch_index] = loss.item()

                # we run backpropagation on the loss variable which repopulates the gradients all the way
                # back through our model to the input layer
                loss.backward()

                # Use the gradients calculated in the backprop that took place in .backwards() to do a new
                # gradient descent step
                optimizer.step()
            
            ## After each epoch -- we should test validation accuracy
            if do_validation:
                if show_validation: print("Epoch: ", epoch)
                validation_scores[epoch] = self.score("Validation", print_score=show_validation)
            
                
        ## Set Loss Matrix for visualizing
        self.set_params(training_losses=losses)
        self.set_params(validation_scores=validation_scores)
        
        return self
        

In [10]:
## Our PyTorch implementation of MLP
from torch.nn import functional as F
class MLP(torch.nn.Module):

    ## the constructor is where we'll define all our layers (input, hidden, and output)
    def __init__(self, hidden=50):

        ## this line creates an instance of our parent (or base) class which in this case
        ## is nn.Module.
        super().__init__()

        ## in the lines below we'll create instance variables and assign them torch.nn Models
        ## in order to create our layers.  You should ordinarily have one variable definition for each layer
        ## in your neural network except for the output layer.  The output layer is defined by the number of
        ## outputs in your last layer. Since we're dealing with simple Artificial Neural Networks, we should
        ## predominantly be using nn.Linear.  
        
        ## definee layer1
        self.l1 = torch.nn.Linear(784, hidden)
        
        
        # initialize layer1
        torch.nn.init.xavier_uniform(self.l1.weight)
        torch.nn.init.constant(self.l1.bias, 0.0)
        
        # define layer2
        self.l2 = torch.nn.Linear(hidden,10)

 
    # forwards takes as a parameter x -- the batch of inputs that we want to feed into our neural network model
    # and returns the output of the model ... i.e. the results of the output layer of the model after forward
    # propagation through our model. practically this means you should call each layer you defined in the
    # constructor in sequence plus any activation functions on each layer.
    def forward(self, x):
     
        # call all our layers on our input (in this case we only need one)
        x = self.l1(x)
        x = F.tanh(x)
        x = self.l2(x)

        # Since we're using Cross Entropy Loss
        # we can return our output directly
        return x



**Ok, lets run with a reg rate of 0.001, learning rate 0.1,batch size 100, 10 epochs, and 100 neurons in the hidden layer**

In [11]:
print("MNIST MLP -- Hidden {}".format(100))
mnist_MLP= ANN(MLP,reg_rate=0.001,learning_rate=0.1,batch_size=100, hidden=100, epochs=10)
mnist_MLP.fit()


In [13]:
mnist_MLP.score(dataset="Validation") 

In [14]:
mnist_MLP.score(dataset="Test") 

Lets change the number of hidden neurons between 50 to 150

In [142]:
# your code here


**Lets try to find the optimal learning rate.**. Try [0.001, 0.01, 0.1, 0.5, 1].

In [145]:
# your code here


**Use the `optimal_learning_rate`  As noted earlier, batch size has less effect, but let's use a size of 100 anyway**

In [146]:
# your code here


In [147]:
mnist_MLP.score(dataset = "Test")

In [148]:
viz_training_loss(mnist_MLP.get_params('training_losses'))

**Great.  Test set accuracy of ~96%.  Let's try some different regularization rates.**

In [ ]:
# your code here